## Notebook for making predictions with deleted constant y samples


In [38]:
import pandas as pd
import sys
from sklearn.model_selection import train_test_split
from catboost import CatBoostRegressor
import numpy as np
from sklearn.ensemble import StackingRegressor
import lightgbm as lgb
from sklearn.neural_network import MLPRegressor
from sklearn.linear_model import LinearRegression

### Reading dataset stored in other notebooks

In [39]:
sys.path.append('../')

%store -r dml

dm = dml

## Getting data

In [40]:
print(dm.data_A.columns)

X_A_train = dm.data_A.iloc[:, 2:]
Y_A_train = dm.data_A.iloc[:, 1]

X_B_train = dm.data_B.iloc[:, 2:]
Y_B_train = dm.data_B.iloc[:, 1]

X_C_train = dm.data_C.iloc[:, 2:]
Y_C_train = dm.data_C.iloc[:, 1]

X_A_test = dm.X_test_estimated_a[dm.X_test_estimated_a.columns.intersection(X_A_train.columns)]
X_B_test = dm.X_test_estimated_b[dm.X_test_estimated_a.columns.intersection(X_A_train.columns)]
X_C_test = dm.X_test_estimated_c[dm.X_test_estimated_a.columns.intersection(X_A_train.columns)]

#X_A_train['direct_rad:W_lag_12']

Index(['date_forecast', 'pv_measurement', 'absolute_humidity_2m:gm3',
       'air_density_2m:kgm3', 'clear_sky_energy_1h:J', 'clear_sky_rad:W',
       'dew_or_rime:idx', 'dew_point_2m:K', 'diffuse_rad:W',
       'diffuse_rad_1h:J', 'direct_rad:W', 'direct_rad_1h:J',
       'effective_cloud_cover:p', 'elevation:m', 'fresh_snow_12h:cm',
       'fresh_snow_1h:cm', 'fresh_snow_24h:cm', 'fresh_snow_3h:cm',
       'fresh_snow_6h:cm', 'is_day:idx', 'is_in_shadow:idx',
       'msl_pressure:hPa', 'precip_5min:mm', 'precip_type_5min:idx',
       'pressure_100m:hPa', 'pressure_50m:hPa', 'prob_rime:p',
       'rain_water:kgm2', 'relative_humidity_1000hPa:p', 'sfc_pressure:hPa',
       'snow_depth:cm', 'snow_drift:idx', 'snow_melt_10min:mm',
       'snow_water:kgm2', 'sun_azimuth:d', 'sun_elevation:d',
       'super_cooled_liquid_water:kgm2', 't_1000hPa:K', 'total_cloud_cover:p',
       'visibility:m', 'wind_speed_10m:ms', 'wind_speed_u_10m:ms',
       'wind_speed_v_10m:ms', 'wind_speed_w_1000hPa:m

In [58]:
# remove special characters from column names
X_A_train.columns = X_A_train.columns.str.replace(':', '_')
X_B_train.columns = X_B_train.columns.str.replace(':', '_')
X_C_train.columns = X_C_train.columns.str.replace(':', '_')

X_A_test.columns = X_A_test.columns.str.replace(':', '_')
X_B_test.columns = X_B_test.columns.str.replace(':', '_')
X_C_test.columns = X_C_test.columns.str.replace(':', '_')

## Catboost models
will try with two models:
-   One for dataset A
-   One for B and C

In [42]:
catModel_a = CatBoostRegressor()
catModel_b = CatBoostRegressor()
catModel_c = CatBoostRegressor()

In [43]:
catModel_a0 = CatBoostRegressor()
catModel_b0 = CatBoostRegressor()
catModel_c0 = CatBoostRegressor()

In [44]:
nn_a = MLPRegressor()
nn_b = MLPRegressor()
nn_c = MLPRegressor()

In [45]:
gbm_a = lgb.LGBMRegressor()
gbm_b = lgb.LGBMRegressor()
gbm_c = lgb.LGBMRegressor()

In [46]:
# meta models
metaModel_a = LinearRegression()
metaModel_b = LinearRegression()
metaModel_c = LinearRegression()

In [47]:
# stacking classifier
stacking_model_a = StackingRegressor(
    estimators=[('cat', catModel_a), ('nn', nn_a), ('gbm', gbm_a)],
    final_estimator=metaModel_a
)

stacking_model_b = StackingRegressor(
    estimators=[('cat', catModel_b), ('nn', nn_b), ('gbm', gbm_b)],
    final_estimator=metaModel_b
)

stacking_model_c = StackingRegressor(
    estimators=[('cat', catModel_c), ('nn', nn_c), ('gbm', gbm_c)],
    final_estimator=metaModel_c
)

In [48]:
# fit stacking models
stacking_model_a.fit(X_A_train, Y_A_train)
stacking_model_b.fit(X_B_train, Y_B_train)
stacking_model_c.fit(X_C_train, Y_C_train)

Learning rate set to 0.067297
0:	learn: 1247.6744460	total: 37.1ms	remaining: 37s
1:	learn: 1186.0260545	total: 41.6ms	remaining: 20.7s
2:	learn: 1130.6434933	total: 45.3ms	remaining: 15.1s
3:	learn: 1078.1808484	total: 49.2ms	remaining: 12.2s
4:	learn: 1029.7823566	total: 52.8ms	remaining: 10.5s
5:	learn: 985.1533261	total: 57.4ms	remaining: 9.51s
6:	learn: 943.6513425	total: 61.4ms	remaining: 8.71s
7:	learn: 906.2632481	total: 65.6ms	remaining: 8.14s
8:	learn: 872.3898611	total: 69.7ms	remaining: 7.68s
9:	learn: 840.7465563	total: 74ms	remaining: 7.32s
10:	learn: 812.1274775	total: 77.8ms	remaining: 7s
11:	learn: 784.7392676	total: 81.9ms	remaining: 6.74s
12:	learn: 760.7637750	total: 86.2ms	remaining: 6.54s
13:	learn: 738.6033910	total: 90.5ms	remaining: 6.37s
14:	learn: 718.7709395	total: 94.7ms	remaining: 6.22s
15:	learn: 700.9002452	total: 99ms	remaining: 6.09s
16:	learn: 684.6465070	total: 103ms	remaining: 5.95s
17:	learn: 669.7781852	total: 107ms	remaining: 5.84s
18:	learn: 655

83:	learn: 502.0415203	total: 404ms	remaining: 4.41s
84:	learn: 501.5113598	total: 408ms	remaining: 4.4s
85:	learn: 501.0798399	total: 413ms	remaining: 4.39s
86:	learn: 500.7192613	total: 418ms	remaining: 4.38s
87:	learn: 500.2373145	total: 422ms	remaining: 4.37s
88:	learn: 499.8466744	total: 427ms	remaining: 4.37s
89:	learn: 499.1939421	total: 431ms	remaining: 4.36s
90:	learn: 498.6641560	total: 436ms	remaining: 4.35s
91:	learn: 498.3434117	total: 440ms	remaining: 4.34s
92:	learn: 497.7481971	total: 444ms	remaining: 4.33s
93:	learn: 497.4321155	total: 448ms	remaining: 4.32s
94:	learn: 497.0505086	total: 453ms	remaining: 4.31s
95:	learn: 496.7569091	total: 458ms	remaining: 4.31s
96:	learn: 496.4050299	total: 462ms	remaining: 4.3s
97:	learn: 496.0678016	total: 466ms	remaining: 4.29s
98:	learn: 495.7401632	total: 470ms	remaining: 4.28s
99:	learn: 495.3954401	total: 475ms	remaining: 4.27s
100:	learn: 495.2168461	total: 480ms	remaining: 4.27s
101:	learn: 494.9583820	total: 484ms	remaining:

StackingRegressor(estimators=[('cat',
                               <catboost.core.CatBoostRegressor object at 0x0000015C7DFA70A0>),
                              ('nn', MLPRegressor()),
                              ('gbm', LGBMRegressor())],
                  final_estimator=LinearRegression())

In [49]:
# fit base models
catModel_a0.fit(X_A_train, Y_A_train)
catModel_b0.fit(X_B_train, Y_B_train)
catModel_c0.fit(X_C_train, Y_C_train)

Learning rate set to 0.067297
0:	learn: 1247.6744460	total: 22.6ms	remaining: 22.6s
1:	learn: 1186.0260545	total: 27.8ms	remaining: 13.9s
2:	learn: 1130.6434933	total: 32ms	remaining: 10.6s
3:	learn: 1078.1808484	total: 36.7ms	remaining: 9.13s
4:	learn: 1029.7823566	total: 40.9ms	remaining: 8.14s
5:	learn: 985.1533261	total: 45ms	remaining: 7.46s
6:	learn: 943.6513425	total: 49.6ms	remaining: 7.04s
7:	learn: 906.2632481	total: 54.1ms	remaining: 6.71s
8:	learn: 872.3898611	total: 58.4ms	remaining: 6.43s
9:	learn: 840.7465563	total: 62.8ms	remaining: 6.22s
10:	learn: 812.1274775	total: 67.7ms	remaining: 6.08s
11:	learn: 784.7392676	total: 72ms	remaining: 5.92s
12:	learn: 760.7637750	total: 76.1ms	remaining: 5.78s
13:	learn: 738.6033910	total: 80.6ms	remaining: 5.68s
14:	learn: 718.7709395	total: 85ms	remaining: 5.58s
15:	learn: 700.9002452	total: 89.8ms	remaining: 5.52s
16:	learn: 684.6465070	total: 94.9ms	remaining: 5.49s
17:	learn: 669.7781852	total: 99.5ms	remaining: 5.43s
18:	learn: 

## Preds

In [54]:
print(X_A_test.columns)

Index(['absolute_humidity_2m:gm3', 'air_density_2m:kgm3',
       'clear_sky_energy_1h:J', 'clear_sky_rad:W', 'dew_or_rime:idx',
       'dew_point_2m:K', 'diffuse_rad:W', 'diffuse_rad_1h:J', 'direct_rad:W',
       'direct_rad_1h:J', 'effective_cloud_cover:p', 'elevation:m',
       'fresh_snow_12h:cm', 'fresh_snow_1h:cm', 'fresh_snow_24h:cm',
       'fresh_snow_3h:cm', 'fresh_snow_6h:cm', 'is_day:idx',
       'is_in_shadow:idx', 'msl_pressure:hPa', 'precip_5min:mm',
       'precip_type_5min:idx', 'pressure_100m:hPa', 'pressure_50m:hPa',
       'prob_rime:p', 'rain_water:kgm2', 'relative_humidity_1000hPa:p',
       'sfc_pressure:hPa', 'snow_depth:cm', 'snow_drift:idx',
       'snow_melt_10min:mm', 'snow_water:kgm2', 'sun_azimuth:d',
       'sun_elevation:d', 'super_cooled_liquid_water:kgm2', 't_1000hPa:K',
       'total_cloud_cover:p', 'visibility:m', 'wind_speed_10m:ms',
       'wind_speed_u_10m:ms', 'wind_speed_v_10m:ms', 'wind_speed_w_1000hPa:ms',
       'direct_rad:W_lag_-23', 'month'

In [59]:
# preds for stacking models
stacking_preds_a = stacking_model_a.predict(X_A_test)
stacking_preds_b = stacking_model_b.predict(X_B_test)
stacking_preds_c = stacking_model_c.predict(X_C_test)

In [60]:
# preds for base models
pred_a = catModel_a0.predict(X_A_test)
pred_b = catModel_b0.predict(X_B_test)
pred_c = catModel_c0.predict(X_C_test)

In [61]:
pred_A_new = np.mean([pred_a, stacking_preds_a], axis=0)
pred_B_new = np.mean([pred_b, stacking_preds_b], axis=0)
pred_C_new = np.mean([pred_c, stacking_preds_c], axis=0)

In [63]:
df_A = pd.DataFrame()

df_A["predict"] = pred_A_new
df_A["location"] = "A"

df_B = pd.DataFrame()

df_B["predict"] = pred_B_new
df_B["location"] = "B"

df_C = pd.DataFrame()

df_C["predict"] = pred_C_new
df_C["location"] = "C"

df_mid = pd.concat([df_A, df_B], ignore_index=True)

df = pd.concat([df_mid, df_C], join="inner", ignore_index=True)

df = df.drop("location", axis=1)

df[df<0] = 0

# NAME THE FILE 
df.to_csv("./ext_preds/stack_3.csv")

df.shape

(2160, 1)